In [1]:
def WriteHeader(f, TaskName):
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=single\n")
    f.write("#SBATCH --ntasks-per-node=26\n")
    f.write("#SBATCH --time=72:00:00\n")
    f.write(f"#SBATCH --job-name={TaskName}\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

In [8]:
def WriteHeader(f, TaskName):
    f.write("#!/bin/bash\n\n")
    f.write("#SBATCH --partition=multiple\n")
    f.write("#SBATCH --nodes=2\n")
    f.write("#SBATCH --ntasks-per-node=40\n")
    f.write("#SBATCH --mem-per-cpu=1125mb\n")
    f.write("#SBATCH --time=72:00:00\n")
    f.write(f"#SBATCH --job-name={TaskName}\n")
    f.write("#SBATCH --error=%x.%j.err\n")
    f.write("#SBATCH --output=%x.%j.out\n")
    f.write("#SBATCH --mail-user=hzhao@teco.edu\n\n")

In [15]:
for epsilon in [0., 0.05, 0.1]:
    with open(f'run_baseline_pNN_{epsilon}_2.sh', 'w') as f:
        WriteHeader(f, f'pNN{epsilon}')
        for ds in range(13):
            for seed in range(10):
                line = f'python3 experiment.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAwareTraining &\n'
                f.write(line)
        f.write("\nwait\n")

In [4]:
for epsilon in [0., 0.05, 0.1]:
    with open(f'run_baseline_PNN_{epsilon}_2.sh', 'w') as f:
        WriteHeader(f, f'pNN{epsilon}')
        for ds in range(13):
            for seed in range(10):
                line = f'python3 experiment.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAwareBaseline &\n'
                f.write(line)
        f.write("\nwait\n")

In [9]:
def write_jobs(f, job_counter, epsilon, ds, seed):
    line = f'python3 experiment.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAwareTraining &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

for epsilon in [0., 0.05, 0.1]:
    # Open the first file for the current epsilon
    f = open(f'run_baseline_pNN_{epsilon}_{file_counter}.sh', 'w')
    WriteHeader(f, f'EAI{epsilon}')

    for ds in range(13):
        for seed in range(10):
            if job_counter >= 80:
                # Close the current file and reset the job counter
                f.write("\nwait\n")
                f.close()
                file_counter += 1
                job_counter = 0
                
                # Open a new file
                f = open(f'run_baseline_pNN_{epsilon}_{file_counter}.sh', 'w')
                WriteHeader(f, f'pNN{epsilon}')
            
            job_counter = write_jobs(f, job_counter, epsilon, ds, seed)

    # If the loop exits but there are still jobs that haven't been written to a closed file
    if not f.closed:
        f.write("\nwait\n")
        f.close()
        job_counter = 0  # Reset the job counter for the next epsilon
        file_counter = 1  # Reset file counter for the next epsilon


In [2]:
def write_jobs(f, job_counter, epsilon, ds, seed):
    line = f'python3 experiment_EA.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAware_EA &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

for epsilon in [0., 0.05, 0.1]:
    # Open the first file for the current epsilon
    f = open(f'run_EA_independent_{epsilon}_{file_counter}.sh', 'w')
    WriteHeader(f, f'EAI{epsilon}')

    for ds in range(13):
        for seed in range(10):
            if job_counter >= 26:
                # Close the current file and reset the job counter
                f.write("\nwait\n")
                f.close()
                file_counter += 1
                job_counter = 0
                
                # Open a new file
                f = open(f'run_EA_independent_{epsilon}_{file_counter}.sh', 'w')
                WriteHeader(f, f'EAI{epsilon}')
            
            job_counter = write_jobs(f, job_counter, epsilon, ds, seed)

    # If the loop exits but there are still jobs that haven't been written to a closed file
    if not f.closed:
        f.write("\nwait\n")
        f.close()
        job_counter = 0  # Reset the job counter for the next epsilon
        file_counter = 1  # Reset file counter for the next epsilon


In [7]:
def write_jobs(f, job_counter, epsilon, ds, seed):
    line = f'python3 experiment_EA_shared.py --DATASET {ds:02d} --SEED {seed:02d} --e_train {epsilon} --projectname VariationAware_EA_shared &\n'
    f.write(line)
    job_counter += 1
    return job_counter

job_counter = 0
file_counter = 1

for epsilon in [0., 0.05, 0.1]:
    # Open the first file for the current epsilon
    f = open(f'run_EA_shared_{epsilon}_{file_counter}.sh', 'w')
    WriteHeader(f, f'EAS{epsilon}')

    for ds in range(13):
        for seed in range(10):
            if job_counter >= 16:
                # Close the current file and reset the job counter
                f.write("\nwait\n")
                f.close()
                file_counter += 1
                job_counter = 0
                
                # Open a new file
                f = open(f'run_EA_shared_{epsilon}_{file_counter}.sh', 'w')
                WriteHeader(f, f'EAS{epsilon}')
            
            job_counter = write_jobs(f, job_counter, epsilon, ds, seed)

    # If the loop exits but there are still jobs that haven't been written to a closed file
    if not f.closed:
        f.write("\nwait\n")
        f.close()
        job_counter = 0  # Reset the job counter for the next epsilon
        file_counter = 1  # Reset file counter for the next epsilon


In [1]:
with open('evaluation_baseline_area.sh', 'w') as f:
    for ds in range(13):
        line = f'sbatch evaluation.py --DATASET {ds}  --areaestimator none --DEVICE cpu\n'
        f.write(line)